In [ ]:
import numpy as np
from sklearn.model_selection import RandomizedSearchCV# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 500, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]# Create the random grid
params = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [ ]:
from main import main
main()

In [ ]:
!pip3 install dask_ml

In [ ]:
import forest

forest.searchBestForest(forest.createRamdomForestGrid(),forest.getLocalDaskCLusterLenov())

In [ ]:
import dask.dataframe as dd
data=dd.read_csv('../ignore/dataPrepared.csv')
data = data.drop(columns='Unnamed: 0').compute()

### paso

In [ ]:
import pandas as pd
data= pd.read_csv('../ignore/data.csv').drop(columns=['x','y','z'])
data.head()
data.dtypes

In [ ]:
from pca import convertToNumber
nDf= convertToNumber(data[['cut','color','clarity']])

In [ ]:
for col in nDf.columns:
    data[col]=nDf[col]

In [ ]:
#data.price=data.price.apply(lambda p:p/10**5)
data.price.describe()

In [ ]:
data.price=data.price.apply(lambda num:num/10**5)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data.drop(columns='price'), data.price, test_size=0.05)

In [ ]:
X_train.columns

In [ ]:
from sklearn.ensemble import RandomForestRegressor
model= RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=110,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=10,
                      min_weight_fraction_leaf=0.0, n_estimators=333,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

model.fit(X_train, y_train)

In [ ]:
print({'model':model,'R**2':model.score(X_test, y_test)})

In [ ]:
test= pd.read_csv('../ignore/test.csv').drop(columns=['x','y','z'])
test.head()

In [ ]:
nDf= convertToNumber(test[['cut','color','clarity']])

In [ ]:
for col in nDf.columns:
    test[col]=nDf[col]

In [ ]:
price=model.predict(test[['carat', 'cut', 'color', 'clarity', 'depth', 'table']])

In [ ]:
subir=pd.DataFrame(price)
subir.columns=['price']
subir.index.name='id'
subir.price=subir.price.apply(lambda p:int(round(p*10**5,0)))
subir.head()

In [ ]:
subir.price=subir.price.apply(lambda p:int(round(p,0)))
subir.head()

In [ ]:
subir.to_csv('../ignore/sample_submission.csv')

In [ ]:
submision=pd.read_csv('../ignore/sample_submission.csv')
submision.head()

In [ ]:
import dask.dataframe as dd
from dask_ml.model_selection import train_test_split

data=dd.read_csv('../ignore/dataPrepared.csv').drop(columns='Unnamed: 0')
X_train, X_test, y_train,y_test = train_test_split(data.drop(columns='price'), data.price, test_size=0.2)
[ele.compute() for ele in [X_train, X_test, y_train,y_test]]